In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/cat-in-the-dat/sample_submission.csv
/kaggle/input/cat-in-the-dat/train.csv
/kaggle/input/cat-in-the-dat/test.csv


In [2]:
submission=pd.read_csv('../input/cat-in-the-dat/sample_submission.csv')
train = pd.read_csv("../input/cat-in-the-dat/train.csv")
test = pd.read_csv("../input/cat-in-the-dat/test.csv")


In [3]:
display(train.head())
display(test.head())


,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,300000,0,0,1,T,Y,Blue,Triangle,Axolotl,Finland,...,9d117320c,3c49b42b8,2,Novice,Warm,j,P,be,5,11
1,300001,0,0,0,T,N,Red,Square,Lion,Canada,...,46ae3059c,285771075,1,Master,Lava Hot,l,A,RP,7,5
2,300002,1,0,1,F,Y,Blue,Square,Dog,China,...,b759e21f0,6f323c53f,2,Expert,Freezing,a,G,tP,1,12
3,300003,0,0,1,T,Y,Red,Star,Cat,China,...,0b6ec68ff,b5de3dcc4,1,Contributor,Lava Hot,b,Q,ke,2,3
4,300004,0,1,1,F,N,Red,Trapezoid,Dog,China,...,f91f3b1ee,967cfa9c9,3,Grandmaster,Lava Hot,l,W,qK,4,11


In [4]:
train.columns

Index(['id', 'bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'nom_0', 'nom_1',
       'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9',
       'ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5', 'day', 'month',
       'target'],
      dtype='object')

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 25 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      300000 non-null  int64 
 1   bin_0   300000 non-null  int64 
 2   bin_1   300000 non-null  int64 
 3   bin_2   300000 non-null  int64 
 4   bin_3   300000 non-null  object
 5   bin_4   300000 non-null  object
 6   nom_0   300000 non-null  object
 7   nom_1   300000 non-null  object
 8   nom_2   300000 non-null  object
 9   nom_3   300000 non-null  object
 10  nom_4   300000 non-null  object
 11  nom_5   300000 non-null  object
 12  nom_6   300000 non-null  object
 13  nom_7   300000 non-null  object
 14  nom_8   300000 non-null  object
 15  nom_9   300000 non-null  object
 16  ord_0   300000 non-null  int64 
 17  ord_1   300000 non-null  object
 18  ord_2   300000 non-null  object
 19  ord_3   300000 non-null  object
 20  ord_4   300000 non-null  object
 21  ord_5   300000 non-null  object
 

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 24 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      200000 non-null  int64 
 1   bin_0   200000 non-null  int64 
 2   bin_1   200000 non-null  int64 
 3   bin_2   200000 non-null  int64 
 4   bin_3   200000 non-null  object
 5   bin_4   200000 non-null  object
 6   nom_0   200000 non-null  object
 7   nom_1   200000 non-null  object
 8   nom_2   200000 non-null  object
 9   nom_3   200000 non-null  object
 10  nom_4   200000 non-null  object
 11  nom_5   200000 non-null  object
 12  nom_6   200000 non-null  object
 13  nom_7   200000 non-null  object
 14  nom_8   200000 non-null  object
 15  nom_9   200000 non-null  object
 16  ord_0   200000 non-null  int64 
 17  ord_1   200000 non-null  object
 18  ord_2   200000 non-null  object
 19  ord_3   200000 non-null  object
 20  ord_4   200000 non-null  object
 21  ord_5   200000 non-null  object
 

In [7]:
print('Dimension of train:',train.shape) 
print('Dimension of test:',test.shape)

Dimension of train: (300000, 25)
Dimension of test: (200000, 24)


****Binary Variables****

In [8]:
unique_values=train.nunique()
unique_values

id        300000
bin_0          2
bin_1          2
bin_2          2
bin_3          2
bin_4          2
nom_0          3
nom_1          6
nom_2          6
nom_3          6
nom_4          4
nom_5        222
nom_6        522
nom_7       1220
nom_8       2215
nom_9      11981
ord_0          3
ord_1          5
ord_2          6
ord_3         15
ord_4         26
ord_5        192
day            7
month         12
target         2
dtype: int64

In [9]:
bin_columns=['bin_0','bin_1','bin_3','bin_4']
binary_data_1=pd.DataFrame(train,columns=bin_columns)
binary_data_2=pd.DataFrame(test,columns=bin_columns)

In [10]:
train['bin_3'] = train['bin_3'].apply(lambda x: 0 if x == 'F' else 1)
train['bin_4'] = train['bin_4'].apply(lambda x: 0 if x == 'N' else 1)

test['bin_3'] = test['bin_3'].apply(lambda x: 0 if x == 'F' else 1)
test['bin_4'] = test['bin_4'].apply(lambda x: 0 if x == 'N' else 1)

In [11]:
train.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,1,1,Green,Triangle,Snake,Finland,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,1,0,1,0,1,1,Green,Trapezoid,Hamster,Russia,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,2,0,0,0,0,1,Blue,Trapezoid,Lion,Russia,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,3,0,1,0,0,1,Red,Trapezoid,Snake,Canada,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,4,0,0,0,0,0,Red,Trapezoid,Lion,Canada,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


****NOMINAL VARIABLES****

In [12]:
nominal=[f'nom_{i}' for i in range(10)]
nominal_data=train.loc[:,nominal]
nominal_data.head()

,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9
0,Green,Triangle,Snake,Finland,Bassoon,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51
1,Green,Trapezoid,Hamster,Russia,Piano,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21
2,Blue,Trapezoid,Lion,Russia,Theremin,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0
3,Red,Trapezoid,Snake,Canada,Oboe,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71
4,Red,Trapezoid,Lion,Canada,Oboe,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7


*********Encoding the nominal features from nom_0 to nom_4 with oneHotEncoding****************

In [13]:
##Now extract the nominal variables for one hot encoding of train and test data.##
one_hot_train = pd.get_dummies(train[['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4']], drop_first = True)

one_hot_test = pd.get_dummies(test[['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4']], drop_first = True)

###Droping the variables###
train.drop(['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4'], axis=1, inplace=True)
test.drop(['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4'], axis=1, inplace=True)


*********Nominal Features for High Cardinality Features with frequency encoder*******************


In [14]:
high_card = ['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9'] 
for col in high_card:
    enc_nom_1 = (train.groupby(col).size()) / len(train)
    train[f'freq_{col}'] = train[col].apply(lambda x : enc_nom_1[x])
    enc_nom_2 = (test.groupby(col).size()) / len(test)
    test[f'freq_{col}'] = test[col].apply(lambda x : enc_nom_2[x])

    '''Droping the variables'''
train.drop(['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9'], axis=1, inplace=True)
test.drop(['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9'], axis=1, inplace=True)          

****ORDINAL ENCODING****

In [15]:
ordinal=[f'ord_{i}' for i in range(6)]
ordinal_data=train.loc[:,ordinal]
ordinal_data.head()



,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5
0,2,Grandmaster,Cold,h,D,kr
1,1,Grandmaster,Hot,a,A,bF
2,1,Expert,Lava Hot,h,R,Jc
3,1,Grandmaster,Boiling Hot,i,D,kW
4,1,Grandmaster,Freezing,a,R,qP


In [16]:
for ord in ordinal:
    print(f'for {ord}: unique values are:{ordinal_data[ord].unique()}')

for ord_0: unique values are:[2 1 3]
for ord_1: unique values are:['Grandmaster' 'Expert' 'Novice' 'Contributor' 'Master']
for ord_2: unique values are:['Cold' 'Hot' 'Lava Hot' 'Boiling Hot' 'Freezing' 'Warm']
for ord_3: unique values are:['h' 'a' 'i' 'j' 'g' 'e' 'd' 'b' 'k' 'f' 'l' 'n' 'o' 'c' 'm']
for ord_4: unique values are:['D' 'A' 'R' 'E' 'P' 'K' 'V' 'Q' 'Z' 'L' 'F' 'T' 'U' 'S' 'Y' 'B' 'H' 'J'
 'N' 'G' 'W' 'I' 'O' 'C' 'X' 'M']
for ord_5: unique values are:['kr' 'bF' 'Jc' 'kW' 'qP' 'PZ' 'wy' 'Ed' 'qo' 'CZ' 'qX' 'su' 'dP' 'aP'
 'MV' 'oC' 'RL' 'fh' 'gJ' 'Hj' 'TR' 'CL' 'Sc' 'eQ' 'kC' 'qK' 'dh' 'gM'
 'Jf' 'fO' 'Eg' 'KZ' 'Vx' 'Fo' 'sV' 'eb' 'YC' 'RG' 'Ye' 'qA' 'lL' 'Qh'
 'Bd' 'be' 'hT' 'lF' 'nX' 'kK' 'av' 'uS' 'Jt' 'PA' 'Er' 'Qb' 'od' 'ut'
 'Dx' 'Xi' 'on' 'Dc' 'sD' 'rZ' 'Uu' 'sn' 'yc' 'Gb' 'Kq' 'dQ' 'hp' 'kL'
 'je' 'CU' 'Fd' 'PQ' 'Bn' 'ex' 'hh' 'ac' 'rp' 'dE' 'oG' 'oK' 'cp' 'mm'
 'vK' 'ek' 'dO' 'XI' 'CM' 'Vf' 'aO' 'qv' 'jp' 'Zq' 'Qo' 'DN' 'TZ' 'ke'
 'cG' 'tP' 'ud' 'tv' 'aM' 'xy' 'lx' '

In [17]:
# ordinal encoding on train data (ord_1,ord_2,ord_3,ord_4)
train.ord_1.replace(to_replace = ['Novice', 'Contributor','Expert', 'Master', 'Grandmaster'],
                         value = [0, 1, 2, 3, 4], inplace = True)

train.ord_2.replace(to_replace = ['Freezing', 'Cold', 'Warm', 'Hot','Boiling Hot', 'Lava Hot'],
                         value = [0, 1, 2, 3, 4, 5], inplace = True)

train.ord_3.replace(to_replace = ['a', 'b', 'c', 'd', 'e', 'f', 'g','h', 'i', 'j', 'k', 'l', 'm', 'n', 'o'],
                         value = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], inplace = True)

train.ord_4.replace(to_replace = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I','J', 'K', 'L', 'M', 'N', 'O', 
                                     'P', 'Q', 'R','S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'],
                         value = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 
                                  22, 23, 24, 25], inplace = True)
test.ord_1.replace(to_replace = ['Novice', 'Contributor','Expert', 'Master', 'Grandmaster'],
                         value = [0, 1, 2, 3, 4], inplace = True)

test.ord_2.replace(to_replace = ['Freezing', 'Cold', 'Warm', 'Hot','Boiling Hot', 'Lava Hot'],
                         value = [0, 1, 2, 3, 4, 5], inplace = True)

test.ord_3.replace(to_replace = ['a', 'b', 'c', 'd', 'e', 'f', 'g','h', 'i', 'j', 'k', 'l', 'm', 'n', 'o'],
                         value = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], inplace = True)

test.ord_4.replace(to_replace = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I','J', 'K', 'L', 'M', 'N', 'O', 
                                       'P', 'Q', 'R','S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'],
                         value = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 
                                  22, 23, 24, 25], inplace = True)


**Encoding "ord_5**"

In [18]:
'''Option 4: Simply sort their values by string of ord_5'''
ord_5 = sorted(list(set(train['ord_5'].values)))
ord_5 = dict(zip(ord_5, range(len(ord_5))))
train.loc[:, 'ord_5'] = train['ord_5'].apply(lambda x: ord_5[x]).astype(float)
test.loc[:, 'ord_5'] = test['ord_5'].apply(lambda x: ord_5[x]).astype(float)

In [19]:
train.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target,freq_nom_5,freq_nom_6,freq_nom_7,freq_nom_8,freq_nom_9
0,0,0,0,0,1,1,2,4,1,7,3,136.0,2,2,0,0.008647,0.003827,0.000803,0.000903,0.000063
1,1,0,1,0,1,1,1,4,3,0,0,93.0,7,8,0,0.002640,0.002807,0.000957,0.000370,0.000043
2,2,0,0,0,0,1,1,2,5,7,17,31.0,7,2,0,0.008413,0.003897,0.001583,0.000927,0.000097
3,3,0,1,0,0,1,1,4,4,8,3,134.0,2,1,1,0.003250,0.003917,0.001473,0.000677,0.000147
4,4,0,0,0,0,0,1,4,0,0,17,158.0,7,8,0,0.006700,0.003857,0.001453,0.000690,0.000117


*****Cyclical Encoding**********

In [20]:
def cyclical_encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

train = cyclical_encode(train, 'day', 7)
test = cyclical_encode(test, 'day', 7) 

train = cyclical_encode(train, 'month', 12)
test = cyclical_encode(test, 'month', 12)

train.drop(['day','month'],inplace=True,axis=1)
test.drop(['day','month'],inplace=True,axis=1)

In [21]:
train_encoded = pd.concat([one_hot_train, train], axis = 1)
test_encoded = pd.concat([one_hot_test, test], axis = 1)

In [22]:
train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 42 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   nom_0_Green       300000 non-null  uint8  
 1   nom_0_Red         300000 non-null  uint8  
 2   nom_1_Polygon     300000 non-null  uint8  
 3   nom_1_Square      300000 non-null  uint8  
 4   nom_1_Star        300000 non-null  uint8  
 5   nom_1_Trapezoid   300000 non-null  uint8  
 6   nom_1_Triangle    300000 non-null  uint8  
 7   nom_2_Cat         300000 non-null  uint8  
 8   nom_2_Dog         300000 non-null  uint8  
 9   nom_2_Hamster     300000 non-null  uint8  
 10  nom_2_Lion        300000 non-null  uint8  
 11  nom_2_Snake       300000 non-null  uint8  
 12  nom_3_China       300000 non-null  uint8  
 13  nom_3_Costa Rica  300000 non-null  uint8  
 14  nom_3_Finland     300000 non-null  uint8  
 15  nom_3_India       300000 non-null  uint8  
 16  nom_3_Russia      30

In [23]:
train_encoded.head()

,nom_0_Green,nom_0_Red,nom_1_Polygon,nom_1_Square,nom_1_Star,nom_1_Trapezoid,nom_1_Triangle,nom_2_Cat,nom_2_Dog,nom_2_Hamster,...,target,freq_nom_5,freq_nom_6,freq_nom_7,freq_nom_8,freq_nom_9,day_sin,day_cos,month_sin,month_cos
0,1,0,0,0,0,0,1,0,0,0,...,0,0.008647,0.003827,0.000803,0.000903,0.000063,9.749279e-01,-0.222521,0.866025,0.500000
1,1,0,0,0,0,1,0,0,0,1,...,0,0.002640,0.002807,0.000957,0.000370,0.000043,-2.449294e-16,1.000000,-0.866025,-0.500000
2,0,0,0,0,0,1,0,0,0,0,...,0,0.008413,0.003897,0.001583,0.000927,0.000097,-2.449294e-16,1.000000,0.866025,0.500000
3,0,1,0,0,0,1,0,0,0,0,...,1,0.003250,0.003917,0.001473,0.000677,0.000147,9.749279e-01,-0.222521,0.500000,0.866025
4,0,1,0,0,0,1,0,0,0,0,...,0,0.006700,0.003857,0.001453,0.000690,0.000117,-2.449294e-16,1.000000,-0.866025,-0.500000


In [24]:
test_encoded.head()

,nom_0_Green,nom_0_Red,nom_1_Polygon,nom_1_Square,nom_1_Star,nom_1_Trapezoid,nom_1_Triangle,nom_2_Cat,nom_2_Dog,nom_2_Hamster,...,ord_5,freq_nom_5,freq_nom_6,freq_nom_7,freq_nom_8,freq_nom_9,day_sin,day_cos,month_sin,month_cos
0,0,0,0,0,0,0,1,0,0,0,...,95.0,0.002635,0.003025,0.001145,0.000565,0.000070,-9.749279e-01,-0.222521,-5.000000e-01,8.660254e-01
1,0,1,0,1,0,0,0,0,0,0,...,61.0,0.006655,0.003035,0.001625,0.000860,0.000125,-2.449294e-16,1.000000,5.000000e-01,-8.660254e-01
2,0,0,0,1,0,0,0,0,1,0,...,172.0,0.005420,0.003860,0.000760,0.000975,0.000050,7.818315e-01,0.623490,-2.449294e-16,1.000000e+00
3,0,1,0,0,1,0,0,1,0,0,...,135.0,0.004290,0.003065,0.000960,0.000270,0.000105,9.749279e-01,-0.222521,1.000000e+00,6.123234e-17
4,0,1,0,0,0,1,0,0,1,0,...,157.0,0.008540,0.002870,0.001410,0.000830,0.000065,-4.338837e-01,-0.900969,-5.000000e-01,8.660254e-01


****Reduce the data set size *****

In [25]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object and col_type.name != 'category' and 'datetime' not in col_type.name:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        elif 'datetime' not in col_type.name:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [26]:
train_encoded = reduce_mem_usage(train_encoded)
test_encoded = reduce_mem_usage(test_encoded)

Memory usage of dataframe is 56.08 MB
Memory usage after optimization is: 21.46 MB
Decreased by 61.7%
Memory usage of dataframe is 35.86 MB
Memory usage after optimization is: 14.11 MB
Decreased by 60.6%


In [27]:
'''Seting X and Y'''
X_train = train_encoded.drop(['id', 'target'],axis = 1)
y_train = train_encoded['target']
X_test = test_encoded.drop(['id'], axis = 1)

In [28]:
print('Input matrix dimension:', X_train.shape)
print('Output vector dimension:',y_train.shape)
print('Test data dimension:',X_test.shape)

Input matrix dimension: (300000, 40)
Output vector dimension: (300000,)
Test data dimension: (200000, 40)


In [29]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 40 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   nom_0_Green       300000 non-null  float16
 1   nom_0_Red         300000 non-null  float16
 2   nom_1_Polygon     300000 non-null  float16
 3   nom_1_Square      300000 non-null  float16
 4   nom_1_Star        300000 non-null  float16
 5   nom_1_Trapezoid   300000 non-null  float16
 6   nom_1_Triangle    300000 non-null  float16
 7   nom_2_Cat         300000 non-null  float16
 8   nom_2_Dog         300000 non-null  float16
 9   nom_2_Hamster     300000 non-null  float16
 10  nom_2_Lion        300000 non-null  float16
 11  nom_2_Snake       300000 non-null  float16
 12  nom_3_China       300000 non-null  float16
 13  nom_3_Costa Rica  300000 non-null  float16
 14  nom_3_Finland     300000 non-null  float16
 15  nom_3_India       300000 non-null  float16
 16  nom_3_Russia      30

In [30]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

X, y = make_classification(random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train,test_size=0.2, random_state=42)
pipe = make_pipeline(StandardScaler(), LogisticRegression())
                     
pipe.fit(X_train, y_train)  # apply scaling on training data
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

pipe.score(X_test, y_test)

0.7471

In [31]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)
sgd_clf.fit(X_train, y_train)

SGDClassifier(random_state=42)

In [32]:
sgd_clf.predict(X_test)

array([0, 0, 1, ..., 1, 0, 0], dtype=int8)

In [33]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring="accuracy")

array([0.6954125, 0.6989375, 0.7166375])

In [34]:
    from sklearn import metrics
    import xgboost as xgb
    model = xgb.XGBClassifier()
    model.fit(X_train, y_train)
    print(); print(model)


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)


In [35]:
expected_y  = y_test
predicted_y = model.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(expected_y,predicted_y)
accuracy


0.74785